In [ ]:
# =====================================================
# Data preparation for SpliceFormer (45k Context)
# =====================================================

import pandas as pd
from pyfaidx import Fasta
from Bio.Seq import Seq
import os
import time
from tqdm import tqdm

# ================= USER CONFIG (ONLY CHANGE THIS) =================
# Đường dẫn lưu file output cho SpliceFormer
OUTPUT_DIR = r"D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceFormer\data"
# =================================================================

# ================= FIXED CONFIG =================
GENOME_PATH = r"D:\Homo_sapiens.GRCh38.dna.primary_assembly.fa"
DATA_FOLDER = r"D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val"

# Cấu hình chuẩn cho SpliceFormer (deCODE genetics)
MODEL_MAX_LEN = 45000 
CONTEXT = MODEL_MAX_LEN // 2  # 22500 nt mỗi bên

FILES = ["test_1_1_1.csv", "test_2_1_1.csv", "test_4_1_1.csv", "test_10_1_1.csv", "test_data.csv"]
# ================================================

def get_sequence_worker(row, fasta_obj):
    """
    Trích xuất chuỗi 45,000 bp.
    Splice site sẽ nằm chính xác tại index 22500.
    """
    try:
        # Cấu trúc ID giả định: gene_chr1_1000_+
        parts = row['id'].split('_')
        chrom, pos, strand = parts[1], int(parts[2]), parts[3]

        # Chuyển về 0-based index
        # Giả sử pos trong ID là vị trí bắt đầu của Intron (Donor) hoặc Exon (Acceptor)
        actual_pos = pos - 1 

        start = actual_pos - CONTEXT
        end = actual_pos + CONTEXT

        # Lấy sequence từ genome (pyfaidx xử lý lazy loading rất tốt)
        # Lưu ý: pyfaidx xử lý biên (negative index) tự động nhưng cần cẩn thận
        if start < 0:
            # Nếu tràn biên trái (âm), lấy từ 0 đến end, rồi pad thêm N vào đầu
            seq_part = str(fasta_obj[chrom][0:end]).upper()
            seq = ("N" * abs(start)) + seq_part
        else:
            seq = str(fasta_obj[chrom][start:end]).upper()

        # Kiểm tra nếu thiếu ở đuôi (cuối nhiễm sắc thể)
        if len(seq) < MODEL_MAX_LEN:
            seq = seq + ("N" * (MODEL_MAX_LEN - len(seq)))

        # Xử lý Reverse Complement nếu là mạch âm (-)
        if strand == '-':
            seq = str(Seq(seq).reverse_complement())

        assert len(seq) == MODEL_MAX_LEN
        return seq

    except Exception as e:
        print(f"⚠️ Error at {row.get('id', 'unknown')}: {e}")
        return "N" * MODEL_MAX_LEN

def diagnose_splice_sites(df, sample_size=5):
    """
    Kiểm tra nhanh xem GT/AG có nằm đúng giữa chuỗi 45k không.
    Center index của chuỗi 45000 là 22500.
    """
    center = MODEL_MAX_LEN // 2
    print(f"\n{'Type':<10} | {'Window center (Idx 22500)':<30} | Found?")
    print("-" * 65)

    for label, name in [(1, 'Donor'), (2, 'Acceptor')]:
        subset = df[df['Splicing_types'] == label]
        if len(subset) == 0:
            continue

        samples = subset.sample(min(sample_size, len(subset)))
        for _, row in samples.iterrows():
            seq = row['sequence']
            # Lấy cửa sổ nhỏ quanh tâm để soi: -2 đến +4
            window = seq[center-2 : center+4] 
            
            # Donor (1) mong đợi GT, Acceptor (2) mong đợi AG
            target = "GT" if label == 1 else "AG"
            
            # Highlight cặp nu ở chính giữa [XY]
            display = f"...{window[:2]}[{window[2:4]}]{window[4:]}..."
            
            # Index 22500 và 22501 phải khớp target
            found = "✅" if window[2:4] == target else "❌"

            print(f"{name:<10} | {display:<30} | {found}")

def run():
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    print(f"[{time.strftime('%H:%M:%S')}] Loading genome (Lazy load)...")
    # sequence_always_upper=True giúp chuẩn hóa data ngay từ đầu
    genome = Fasta(GENOME_PATH, sequence_always_upper=True)

    for file_name in FILES:
        input_path = os.path.join(DATA_FOLDER, file_name)
        output_path = os.path.join(OUTPUT_DIR, file_name)

        if not os.path.exists(input_path):
            print(f"❌ Skip {file_name} (not found)")
            continue

        start_time = time.time()
        df = pd.read_csv(input_path)

        # Sắp xếp theo Chromosome và Position để ổ cứng đọc tuần tự -> Tăng tốc độ cực lớn
        # Tạo cột tạm để sort
        try:
            split_data = df['id'].str.split('_', expand=True)
            df['_tmp_chr'] = split_data[1]
            df['_tmp_pos'] = split_data[2].astype(int)
            df = df.sort_values(['_tmp_chr', '_tmp_pos']).reset_index(drop=True)
        except Exception:
            print("⚠️ Warning: Could not sort by ID. Processing unsorted (slower).")

        print(f"\n🚀 Processing {file_name} | {len(df)} rows | Length: {MODEL_MAX_LEN}")

        # Dùng List Comprehension + tqdm sẽ nhanh hơn apply một chút
        sequences = []
        # Chuyển dataframe thành list dict để iterate nhanh hơn
        rows = df.to_dict('records')
        
        for row in tqdm(rows, desc="Extracting 45k seq"):
            sequences.append(get_sequence_worker(row, genome))

        df['sequence'] = sequences

        # Validate dữ liệu
        diagnose_splice_sites(df)

        # Dọn dẹp và lưu
        if '_tmp_chr' in df.columns:
            df.drop(columns=['_tmp_chr', '_tmp_pos'], inplace=True)
            
        # Lưu CSV
        # Lưu ý: File CSV sẽ rất nặng do chuỗi 45k. 
        df.to_csv(output_path, index=False)

        elapsed = time.time() - start_time
        print(f"✅ Saved to: {output_path}")
        print(f"⚡ Speed: {len(df)/elapsed:.2f} seq/s")

    print("\n🎯 DONE – Data ready for SpliceFormer (45k Context)")

# ================= RUN =================
if __name__ == "__main__":
    run()

[23:27:52] Loading genome...

🚀 Processing test_1_1_1.csv | 26310 rows


Extracting: 100%|██████████| 26310/26310 [00:01<00:00, 19478.59it/s]



Type       | Window (-2:+2)       | Found?
-------------------------------------------------------
Donor      | GG[TG]AG             | ❌
Donor      | AT[GT]GA             | ✅
Donor      | GG[TA]AC             | ❌
Donor      | GG[TT]AG             | ❌
Donor      | GG[GT]AC             | ✅
Acceptor   | AG[AA]CC             | ❌
Acceptor   | AA[GG]AG             | ❌
Acceptor   | AG[GT]TG             | ❌
Acceptor   | AA[GT]GC             | ❌
Acceptor   | CA[GC]AA             | ❌
✅ Saved: D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceTransformer\data\test_1_1_1.csv
⚡ Speed: 8509.29 seq/s

🚀 Processing test_2_1_1.csv | 35132 rows


Extracting: 100%|██████████| 35132/35132 [00:01<00:00, 26232.96it/s]



Type       | Window (-2:+2)       | Found?
-------------------------------------------------------
Donor      | AG[GT]AT             | ✅
Donor      | AG[GT]AA             | ✅
Donor      | AA[GT]AA             | ✅
Donor      | AG[GT]AC             | ✅
Donor      | AT[GT]AA             | ✅
Acceptor   | AG[GT]GG             | ❌
Acceptor   | AG[CC]GG             | ❌
Acceptor   | CA[GG]TT             | ❌
Acceptor   | AG[GG]AC             | ❌
Acceptor   | TA[GA]TC             | ❌
✅ Saved: D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceTransformer\data\test_2_1_1.csv
⚡ Speed: 9764.73 seq/s

🚀 Processing test_4_1_1.csv | 52776 rows


...

Extracting: 100%|██████████| 105708/105708 [00:04<00:00, 26223.29it/s]



Type       | Window (-2:+2)       | Found?
-------------------------------------------------------
Donor      | CA[GT]GA             | ✅
Donor      | TG[GT]GA             | ✅
Donor      | GG[TA]AG             | ❌
Donor      | GG[GT]GG             | ✅
Donor      | GG[TC]TG             | ❌
Acceptor   | AG[AG]TG             | ✅
Acceptor   | AA[GG]AA             | ❌
Acceptor   | CA[GA]TC             | ❌
Acceptor   | CA[GA]CT             | ❌
Acceptor   | CA[GG]TG             | ❌
✅ Saved: D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceTransformer\data\test_10_1_1.csv
⚡ Speed: 9681.08 seq/s

🚀 Processing test_data.csv | 938297 rows


Extracting: 100%|██████████| 938297/938297 [00:36<00:00, 26044.40it/s]



Type       | Window (-2:+2)       | Found?
-------------------------------------------------------
Donor      | GG[TG]TG             | ❌
Donor      | AG[GT]AA             | ✅
Donor      | AC[GT]AA             | ✅
Donor      | AG[TT]CT             | ❌
Donor      | GG[TT]CT             | ❌
Acceptor   | TA[GG]GC             | ❌
Acceptor   | CA[GA]TC             | ❌
Acceptor   | TA[GG]GA             | ❌
Acceptor   | CA[GA]AA             | ❌
Acceptor   | AG[AC]AA             | ❌


KeyboardInterrupt: 